In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import csv
import pandas as pd

In [0]:
sess = tf.InteractiveSession()
model = load_model('InceptionV3_87acc_80ep_32cl.hdf5')

In [0]:
#Preprocess image, calling the predic method, find the winner class and match with lable
def predict_class(model, image, show_debug = False):
    img = np.expand_dims(image, axis=0)
    img = img / 255  
    pred = model.predict(img)
    index = np.argmax(pred)
    food_list.sort()
    pred_value = food_list[index]
    fill_dict(pred_value)
    if show_debug:
      plt.imshow(img[0])                           
      plt.axis('off')
      plt.title(pred_value)
      plt.show()

In [0]:
#Fill the dictionary with lables
def fill_dict(key):
  if key in classes:
    classes[key] += 1
  else:
    classes[key] = 1

In [0]:
# Find the winner lables
def winner():
  winners = []
  for k,v in classes.items():
    if v > 3:
      winners.append(k)
  return winners
      #print("On the plate is " + k)

In [0]:
def selecting(sliding):
  global df_global
  for n in range(10,40,10):
    for m in range(80,110,10):
      ksize_rows, ksize_cols = int((sliding[0] * n)/100), int((sliding[1] * n)/100)
      strides_rows, strides_cols =  int((ksize_rows * m)/100), int((ksize_cols * m)/100)

      classes['slidind_window'] = ksize_rows
      classes['strides'] = strides_rows

      ksizes = [1, ksize_rows, ksize_cols, 1] # The size of sliding window

      strides = [1, strides_rows, strides_cols, 1]# How far the centers of 2 consecutive patches are in the image

      rates = [1, 1, 1, 1] # sample pixel consecutively

      padding='VALID' # or 'SAME'

      image_expanded = tf.expand_dims(image, 0)
      image_patches = tf.extract_image_patches(image_expanded, ksizes, strides, rates, padding)

      shapes = sess.run(tf.shape(image_patches))
      nr, nc = shapes[1], shapes[2]
      classes['patches_number'] = nr * nc

      for i in range(nr):
        for j in range(nc):
          x = sess.run(tf.reshape(image_patches[0,i,j,], [ksize_rows, ksize_cols, 3]))
          x = cv2.resize(x, dsize=(300, 300), interpolation=cv2.INTER_CUBIC)
          predict_class(model, x, False)
      print(classes)
      df_local  = pd.DataFrame([classes], columns=classes.keys())
      df_global = df_global.append(df_local, sort=False)
      #print(df_global)
      classes.clear()

In [31]:
#Getting patches from the initial image
%%time
classes = {}
df_global = pd.DataFrame()
image_file = '4classes.JPg'
im = Image.open(image_file)
img_raw = tf.read_file(image_file)


food_list = ['apple pie','baby back ribs','bread pudding','breakfast burrito','caesar salad','chicken curry','chicken wings','club sandwich','cup cakes','dumplings','eggs benedict','filet mignon','fish and chips','french fries','french toast','greek salad','grilled cheese sandwich','grilled salmon','hamburger','hot dog','ice cream','macaroni and cheese','macarons','omelette','pancakes','pizza','pork chop','prime rib','risotto','spaghetti bolognese','spaghetti carbonara','steak']

image = tf.image.decode_image(img_raw).eval()
image = tf.image.resize_images(image, [300, 300]).eval()
sliding = sess.run(tf.shape(image))
selecting(sliding)
df_global.to_csv('Steak.csv', sep='\t')


{'slidind_window': 30, 'strides': 24, 'patches_number': 144, 'pizza': 29, 'apple pie': 2, 'omelette': 2, 'pork chop': 7, 'bread pudding': 5, 'grilled salmon': 6, 'chicken wings': 4, 'risotto': 40, 'french fries': 12, 'ice cream': 3, 'hot dog': 4, 'grilled cheese sandwich': 11, 'chicken curry': 8, 'baby back ribs': 1, 'macaroni and cheese': 1, 'pancakes': 6, 'macarons': 1, 'cup cakes': 2}
{'slidind_window': 30, 'strides': 27, 'patches_number': 121, 'pizza': 30, 'apple pie': 2, 'grilled cheese sandwich': 9, 'pork chop': 4, 'bread pudding': 9, 'chicken wings': 4, 'hot dog': 3, 'risotto': 35, 'baby back ribs': 2, 'macaroni and cheese': 2, 'chicken curry': 4, 'french fries': 8, 'grilled salmon': 3, 'ice cream': 1, 'pancakes': 5}
{'slidind_window': 30, 'strides': 30, 'patches_number': 100, 'pizza': 28, 'pork chop': 3, 'grilled cheese sandwich': 12, 'bread pudding': 2, 'french fries': 6, 'risotto': 29, 'hot dog': 2, 'chicken wings': 2, 'chicken curry': 6, 'grilled salmon': 2, 'ice cream': 2, 

In [32]:
df_global.head(100)

,slidind_window,strides,patches_number,pizza,apple pie,omelette,pork chop,bread pudding,grilled salmon,chicken wings,risotto,french fries,ice cream,hot dog,grilled cheese sandwich,chicken curry,baby back ribs,macaroni and cheese,pancakes,macarons,cup cakes,caesar salad,steak,club sandwich,fish and chips
0,30,24,144,29,2.0,2.0,7,5.0,6.0,4.0,40.0,12.0,3.0,4.0,11.0,8,1.0,1.0,6.0,1.0,2.0,NaN,NaN,NaN,NaN
0,30,27,121,30,2.0,NaN,4,9.0,3.0,4.0,35.0,8.0,1.0,3.0,9.0,4,2.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
0,30,30,100,28,NaN,1.0,3,2.0,2.0,2.0,29.0,6.0,2.0,2.0,12.0,6,NaN,1.0,3.0,NaN,NaN,1.0,NaN,NaN,NaN
0,60,48,36,4,1.0,NaN,4,2.0,NaN,NaN,1.0,4.0,NaN,6.0,4.0,5,NaN,2.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0
0,60,54,25,2,NaN,2.0,6,1.0,1.0,2.0,1.0,NaN,NaN,1.0,3.0,3,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0,NaN
0,60,60,25,2,NaN,1.0,4,1.0,NaN,NaN,2.0,2.0,1.0,2.0,4.0,4,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN
0,90,72,9,1,NaN,1.0,2,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
0,90,81,9,1,NaN,1.0,1,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN
0,90,90,9,1,NaN,1.0,2,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,2,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df_global['target'] = (df_global['pork chop']+ df_global['greek salad'] + df_global['omelette'])/ df_global['patches_number']

In [24]:
df_global[['slidind_window','strides','patches_number','target']].sort_values(by='target', ascending=False)

,slidind_window,strides,patches_number,target
0,150,150,4,1.000000
0,120,72,9,1.000000
0,150,135,4,1.000000
0,150,120,4,1.000000
0,150,105,4,1.000000
0,150,90,4,1.000000
0,150,75,9,1.000000
0,120,120,4,1.000000
0,120,108,4,1.000000
0,120,96,4,1.000000
